# Text Analysis Assignment #3
#### Names: Ali Daanesh Sayed (as92998), Bhavna Kaparaju (bgk378), Callie Gilmore (cgg756), Dawson Cook (dcc2436), India Lindsay (igl257)
#### Section: 8:30 am

## Insta Scraper

In [3]:
# Imports and Installs
!pip install instaloader
import instaloader
import pandas as pd 
import time
import os

  Created wheel for instaloader: filename=instaloader-4.5.3-py3-none-any.whl size=50848 sha256=459bdbe5e6af1b7776162e0a2a43006f96cb0ad12191e71fe855189863a472e8
  Stored in directory: c:\users\bhavn\appdata\local\pip\cache\wheels\00\cb\d8\6add73b3101ae585a6d6e93904320f69fd0508fe46e5a98019
Successfully built instaloader


C:\Users\bhavn\Anaconda3\lib\site-packages\win_unicode_console\__init__.py:31: RuntimeWarning: sys.stdin.encoding == 'cp1252', whereas sys.stdout.encoding == 'UTF-8', readline hook consumer may assume they are the same
  readline_hook.enable(use_pyreadline=use_pyreadline)


In [ ]:
# Getting Zara
L = instaloader.Instaloader()
user_name = 'zara'
profile = instaloader.Profile.from_username(L.context, user_name)

# Converting to DF
posts_df = pd.DataFrame(columns=['num_comments', 'num_likes', 'caption', 'image_url', 'is_video'])

# URL of Zara's Instagram
url = 'https://www.instagram.com/p/{}/'

# Grabbing 700 posts just in case
posts = profile.get_posts()
posts_scrape = 700
number = 0 

In [ ]:
# Scraping # of comments, # of likes, caption, image_url, is_video
for post in posts:
    if number < posts_scrape:
        number += 1
        posts_df = posts_df.append({'num_comments': post.comments,
                                    'num_likes': post.likes,
                                    'caption': post.caption,
                                    'image_url': post.url, 
                                    'is_video': post.is_video}, ignore_index=True)
        posts_df.is_video.astype('bool')
        if posts_df[posts_df.is_video == False].shape[0] == posts_scrape:
            number = posts_scrape      
    else:
        break

In [ ]:
# Only grabbing posts by dropping any values where is_video column is true
posts_df = posts_df[posts_df['is_video'] == False]
# Dropping is_video column as it is no longer  needed
posts_df = posts_df.drop('is_video', 1)
# Exporting as CSV
posts_df.to_csv('insta.csv', index=False)

In [ ]:
posts_df = posts_df.reset_index(drop=True)
posts_df.head(5)

,num_comments,num_likes,caption,image_url
0,421,73609,FW20 Campaign. Kids Collection\nCreative Direc...,https://scontent-dfw5-2.cdninstagram.com/v/t51...
1,126,38690,FW20 Campaign. Kids Collection\nCreative Direc...,https://scontent-dfw5-2.cdninstagram.com/v/t51...
2,196,56404,FW20 Campaign. Kids Collection\nCreative Direc...,https://scontent-dfw5-2.cdninstagram.com/v/t51...
3,106,34923,FW20 Campaign. Man Collection\nCreative Direct...,https://scontent-dfw5-2.cdninstagram.com/v/t51...
4,189,34424,FW20 Campaign. Man Collection\nCreative Direct...,https://scontent-dfw5-2.cdninstagram.com/v/t51...


## Google Vision

In [1]:
from google.colab import files 
uploaded = files.upload()

Saving insta_labels.csv to insta_labels.csv


In [5]:
posts_df = pd.read_csv("insta_labels.csv")

In [ ]:
#!pip install google-cloud-vision

In [ ]:
from google.cloud import vision
import os

"""Initialize Environment Variables to enable authentication with google Vision API"""
#credential_path = r"C:\Users\india\Documents\text-assignment3-94cea931a4be.json" 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credential_path

client = vision.ImageAnnotatorClient()
image = vision.Image() 

"""storing urls for each image and file names"""
file_name=[]
labels=[]
image_paths = []
counter=0

for each_img in posts_df.image_url:
  image_paths.append(each_img)
  file_name.append(str(posts_df.image_url.index[counter]+1) + ".jpg")
  counter+=1

In [ ]:
def getlabelsforRemoteImage(uri):
    """Detects labels in the file located in Google Cloud Storage or on the
    Web."""
    labels_list=[]
    image.source.image_uri = uri

    response = client.label_detection(image=image)
    labels = response.label_annotations
    
    for label in labels:
        labels_list.append(label.description)
    print(uri)
    print(labels_list)
    return(labels_list)

"""grabbing photo labels for each image url in our dataset"""
for i in image_paths:
    labels.append(getlabelsforRemoteImage(i))

In [ ]:
posts_df['labels']=pd.Series(labels) #storing labels in our dataframe


## Task A

In [ ]:
# Scaling Likes and Comments
posts_df['scaled_likes'] = posts_df['num_likes'] / posts_df['num_likes'].max()
posts_df['scaled_comms'] = posts_df['num_comments'] / posts_df['num_comments'].max()

# Creating Engagement Score
posts_df['engagement_score'] = .4 * posts_df['scaled_likes'] + .6 * posts_df['scaled_comms']

# Qualitative 'High' or 'Low'
def engagement_qual(eng_score):
    if eng_score > posts_df['engagement_score'].median():
        return 'High'
    else:
        return 'Low'

# Binary 1 or 0 to perform other tasks
def engagement_bin(eng_score):
    if eng_score > posts_df['engagement_score'].median():
        return 1
    else:
        return 0

In [ ]:
#Creating Engagement Columns
posts_df['engagement'] = posts_df['engagement_score'].apply(engagement_bin)
posts_df['engagement_qual'] = posts_df['engagement_score'].apply(engagement_qual)

In [7]:
posts_df.head()

,num_comments,num_likes,caption,image_url,labels,scaled_likes,scaled_comms,engagement_score,engagement,engagement_qual
0,405,73222,FW20 Campaign. Kids Collection\nCreative Direc...,https://scontent-iad3-1.cdninstagram.com/v/t51...,"['Clothing', 'Fashion', 'Outerwear', 'Fur', 'S...",0.249823,0.124769,0.174791,1,High
1,124,38537,FW20 Campaign. Kids Collection\nCreative Direc...,https://scontent-iad3-1.cdninstagram.com/v/t51...,"['Sky', 'Darkness', 'Room', 'Adventure game', ...",0.131483,0.038201,0.075514,0,Low
2,194,56141,FW20 Campaign. Kids Collection\nCreative Direc...,https://scontent-iad3-1.cdninstagram.com/v/t51...,"['Cool', 'Fashion', 'Jeans', 'Sitting', 'Denim...",0.191545,0.059766,0.112478,0,Low
3,104,34764,FW20 Campaign. Man Collection\nCreative Direct...,https://scontent-iad3-1.cdninstagram.com/v/t51...,"['Hair', 'Face', 'Hairstyle', 'Eyebrow', 'Fore...",0.118610,0.032039,0.066668,0,Low
4,187,34312,FW20 Campaign. Man Collection\nCreative Direct...,https://scontent-iad3-1.cdninstagram.com/v/t51...,"['Snapshot', 'Standing', 'Hand', 'Arm', 'Human...",0.117068,0.057609,0.081393,0,Low


In [ ]:
posts_df.to_csv('insta_labels.csv', index=False) #saving to csv

## Task B
Using TF-IDF scores within a logistic regression to predict engagement.

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


def predicting_eng(X,y):
  """ logistic regression model using 80% train, 20% test split"""
  X_train, X_test, y_train,y_test = train_test_split(X,y,test_size= 0.2, random_state=1)

  """initialize TFIDFVectorizer"""
  tfidf_vectorizer = TfidfVectorizer(stop_words='english',use_idf=True)
  fitted_vectorizer=tfidf_vectorizer.fit(X_train)
  X_train_tfidf=fitted_vectorizer.transform(X_train)
  X_test_tfidf = tfidf_vectorizer.transform(X_test)

  """fitting logistic regression model to data """

  model = LogisticRegression()
  model.fit(X_train_tfidf,y_train)

  """ Predicting on test data""" 
  y_fitted = model.predict(X_test_tfidf)

  """confusion matrix"""
  print(confusion_matrix(y_test,y_fitted))
  
  """classification report"""
  print(classification_report(y_test,y_fitted))


### Predicting engagement using only image labels

In [9]:
X = posts_df['labels']
y = posts_df['engagement']
print("Confusion Matrix and Classification Report: ")
predicting_eng(X,y)

Confusion Matrix and Classification Report: 
[[32 25]
 [17 40]]
              precision    recall  f1-score   support

           0       0.65      0.56      0.60        57
           1       0.62      0.70      0.66        57

    accuracy                           0.63       114
   macro avg       0.63      0.63      0.63       114
weighted avg       0.63      0.63      0.63       114



### Predicting engagement using only captions

In [10]:
X = posts_df['caption']
y = posts_df['engagement']
print("Confusion Matrix and Classification Report: ")
predicting_eng(X,y)

Confusion Matrix and Classification Report: 
[[38 19]
 [16 41]]
              precision    recall  f1-score   support

           0       0.70      0.67      0.68        57
           1       0.68      0.72      0.70        57

    accuracy                           0.69       114
   macro avg       0.69      0.69      0.69       114
weighted avg       0.69      0.69      0.69       114



### Predicting engagement using a combination of captions and image labels

In [11]:
posts_df['caption_labels'] = posts_df['caption'] + ' ' + posts_df['labels']
X = posts_df['caption_labels']
y = posts_df['engagement']
print("Confusion Matrix and Classification Report: ")
predicting_eng(X,y)

Confusion Matrix and Classification Report: 
[[35 22]
 [14 43]]
              precision    recall  f1-score   support

           0       0.71      0.61      0.66        57
           1       0.66      0.75      0.70        57

    accuracy                           0.68       114
   macro avg       0.69      0.68      0.68       114
weighted avg       0.69      0.68      0.68       114



### Interpreting Our Model

The engagement score was calculated by taking a weighted percentage of the number of likes and comments on a post. We set the engagement score to be equal to 1 if the users level of engagement with a post was greater than the median level for all posts. It was set to 0 if it was less than this value. This allowed our dataset to be equally balanced between the two engagement classes.

When predicting engagement using only image labels, our model correctly classified 65% of posts that had an engagement of 0 and 62% that had an engagement of 1. When using only captions, we correctly classified 70% of posts with an engagement score of 0 and 68% with an engagement of 1. When combining the two, we correctly classified 71% of posts with low engagement and 66% of posts with high engagement.

From these results, we can infer that using captions to predict engagement greatly increased our model's accuracy. Using both captions and labels helped improve the model's accuracy for posts with low engagement but only slightly. It decreased the model's predictive ability for posts with high engagement. This suggests that captions are very influential on a user's engagement with a post and carry more weight than the topics depicted within the image. Zara will be able to increase user response to posts by focusing on creating engaging captions.

## Task C

In [7]:
!pip install instaloader
!pip install google-cloud
!pip install google-cloud-vision
!pip install pyLDAvis 
!pip install gensim

  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107102 sha256=436c35862c742e4f6a590b8c30dfdca9de826224db166d7ed063ebec2626d79c
  Stored in directory: c:\users\bhavn\appdata\local\pip\cache\wheels\83\a6\12\bf3c1a667bde4251be5b7a3368b2d604c9af2105b5c1cb1870
Successfully built smart-open
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.15
    Uninstalling Cython-0.29.15:
      Successfully uninstalled Cython-0.29.15


In [15]:
import instaloader
from instaloader import Profile
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from google.cloud import vision
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
import os
import ast
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer 
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
import pyLDAvis
import pyLDAvis.gensim

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bhavn\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [16]:
posts_df = pd.read_csv('insta_labels.csv')

In [17]:
posts_df['index'] = posts_df.index

In [18]:
posts_df.head()

,num_comments,num_likes,caption,image_url,labels,scaled_likes,scaled_comms,engagement_score,engagement,engagement_qual,index
0,405,73222,FW20 Campaign. Kids Collection\nCreative Direc...,https://scontent-iad3-1.cdninstagram.com/v/t51...,"['Clothing', 'Fashion', 'Outerwear', 'Fur', 'S...",0.249823,0.124769,0.174791,1,High,0
1,124,38537,FW20 Campaign. Kids Collection\nCreative Direc...,https://scontent-iad3-1.cdninstagram.com/v/t51...,"['Sky', 'Darkness', 'Room', 'Adventure game', ...",0.131483,0.038201,0.075514,0,Low,1
2,194,56141,FW20 Campaign. Kids Collection\nCreative Direc...,https://scontent-iad3-1.cdninstagram.com/v/t51...,"['Cool', 'Fashion', 'Jeans', 'Sitting', 'Denim...",0.191545,0.059766,0.112478,0,Low,2
3,104,34764,FW20 Campaign. Man Collection\nCreative Direct...,https://scontent-iad3-1.cdninstagram.com/v/t51...,"['Hair', 'Face', 'Hairstyle', 'Eyebrow', 'Fore...",0.118610,0.032039,0.066668,0,Low,3
4,187,34312,FW20 Campaign. Man Collection\nCreative Direct...,https://scontent-iad3-1.cdninstagram.com/v/t51...,"['Snapshot', 'Standing', 'Hand', 'Arm', 'Human...",0.117068,0.057609,0.081393,0,Low,4


In [19]:
data_labels = posts_df[['labels']]
data_labels['index'] = data_labels.index
print(len(data_labels))
data_labels[:10]

570


C:\Users\bhavn\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,labels,index
0,"['Clothing', 'Fashion', 'Outerwear', 'Fur', 'S...",0
1,"['Sky', 'Darkness', 'Room', 'Adventure game', ...",1
2,"['Cool', 'Fashion', 'Jeans', 'Sitting', 'Denim...",2
3,"['Hair', 'Face', 'Hairstyle', 'Eyebrow', 'Fore...",3
4,"['Snapshot', 'Standing', 'Hand', 'Arm', 'Human...",4
5,"['Clothing', 'Fashion', 'Outerwear', 'Beige', ...",5
6,"['People in nature', 'Photograph', 'Black-and-...",6
7,"['Photograph', 'Standing', 'People', 'Suit', '...",7
8,"['Cool', 'Human', 'Font', 'Fur', 'Outerwear', ...",8
9,"['Photograph', 'Clothing', 'Formal wear', 'Sta...",9


### Stemming, Lemmatizing and Pre-processing

In [20]:
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
            result.append(lemmatize_stemming(token))
    return result

In [21]:
doc_sample = data_labels[data_labels['index'] == 200].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

processed_docs = data_labels['labels'].map(preprocess)
processed_docs[:10]

original document: 
["['People',", "'Social", "group',", "'Youth',", "'Fun',", "'Friendship',", "'Smile',", "'Dance',", "'Child',", "'Leg',", "'Happy']"]


 tokenized and lemmatized document: 
['people', 'social', 'group', 'youth', 'fun', 'friendship', 'smile', 'dance', 'child', 'leg', 'happy']


0    [clothe, fashion, outerwear, fur, street, fash...
1    [sky, darkness, room, adventure, game, music, ...
2      [cool, fashion, jeans, sit, denim, shoe, style]
3    [hair, face, hairstyle, eyebrow, forehead, chi...
4    [snapshot, stand, hand, arm, human, photograph...
5    [clothe, fashion, outerwear, beige, fashion, d...
6    [people, nature, photograph, black, white, sta...
7    [photograph, stand, people, suit, black, white...
8    [cool, human, font, fur, outerwear, textile, a...
9    [photograph, clothe, formal, wear, stand, suit...
Name: labels, dtype: object

### Bag of words

In [22]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 clothe
1 coat
2 design
3 fashion
4 fur
5 headgear
6 knee
7 outerwear
8 photography
9 street
10 adventure


In [23]:
dictionary.filter_extremes(no_below=7, no_above=0.5, keep_n=100000)

**Gensim doc2bow**

In [24]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[310]

processed_docs[310]

#To output words and their corresponding frequency

#bow_doc_310 = bow_corpus[310]
#for i in range(len(bow_doc_310)):
    #print("Word {} (\"{}\") appears {} time.".format(bow_doc_310[i][0], 
     #                                          dictionary[bow_doc_310[i][0]], 
#bow_doc_310[i][1]))
#

['sit',
 'beauty',
 'leg',
 'fashion',
 'footwear',
 'photo',
 'shoot',
 'furniture',
 'blond',
 'jeans',
 'photography']

### Running LDA using bag of words

In [25]:
np.random.seed(2020)
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=4, id2word=dictionary, passes=8, workers=2)

In [26]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.124*"hair" + 0.070*"beauty" + 0.050*"hairstyle" + 0.047*"model" + 0.038*"skin" + 0.038*"long" + 0.034*"face" + 0.034*"shoot" + 0.034*"photo" + 0.032*"fashion"
Topic: 1 
Words: 0.074*"black" + 0.073*"photography" + 0.072*"white" + 0.068*"fashion" + 0.041*"blue" + 0.035*"monochrome" + 0.032*"clothe" + 0.032*"outerwear" + 0.030*"stand" + 0.030*"photograph"
Topic: 2 
Words: 0.102*"fashion" + 0.072*"leg" + 0.046*"photography" + 0.041*"clothe" + 0.039*"footwear" + 0.034*"outerwear" + 0.029*"joint" + 0.028*"shoulder" + 0.027*"human" + 0.027*"sit"
Topic: 3 
Words: 0.100*"fashion" + 0.091*"clothe" + 0.070*"outerwear" + 0.058*"sleeve" + 0.051*"neck" + 0.047*"dress" + 0.045*"shoulder" + 0.033*"design" + 0.027*"shirt" + 0.026*"pattern"


### Top-10 words loaded for each topic

In [27]:
x=lda_model.show_topics()

def Convert(string): 
    li = list(string.split("  ")) 
    return li 

twords={}
for topic,word in x:
    twords[topic]=Convert(re.sub('[^A-Za-z ]+','', word))
    print('Topic words:',topic,'\nWords:',twords[topic])
    
terms_df = pd.DataFrame([twords])
terms_df = terms_df.transpose()
terms_df.index.name = 'Topic'
terms_df.columns = ['Words']
terms_df.to_csv('words_loading_on_topics.csv')

Topic words: 0 
Words: ['hair', 'beauty', 'hairstyle', 'model', 'skin', 'long', 'face', 'shoot', 'photo', 'fashion']
Topic words: 1 
Words: ['black', 'photography', 'white', 'fashion', 'blue', 'monochrome', 'clothe', 'outerwear', 'stand', 'photograph']
Topic words: 2 
Words: ['fashion', 'leg', 'photography', 'clothe', 'footwear', 'outerwear', 'joint', 'shoulder', 'human', 'sit']
Topic words: 3 
Words: ['fashion', 'clothe', 'outerwear', 'sleeve', 'neck', 'dress', 'shoulder', 'design', 'shirt', 'pattern']


### Performance Evaluation
This was for manual verification wherein we changed index number everytime manually and checked topic generated for a set of labels

In [30]:
processed_docs[301]

['black',
 'white',
 'eyewear',
 'black',
 'white',
 'beauty',
 'monochrome',
 'glass',
 'cool',
 'stand',
 'shoulder']

In [31]:
for index, score in sorted(lda_model[bow_corpus[301]], key=lambda tup: -1*tup[1]):
    print("\nScore {} \nTopic: {}".format(score, index))


Score 0.6489116549491882 
Topic: 1

Score 0.3078915774822235 
Topic: 3

Score 0.021746888756752014 
Topic: 0

Score 0.02144993282854557 
Topic: 2


### Topic Weights

In [32]:
for i in range(0,len(bow_corpus)):
    for index, score in sorted(lda_model[bow_corpus[i]], key=lambda tup: -1*tup[1]):
        arr = "Topic "+ str(index)
        data_labels.loc[i,arr]= score

C:\Users\bhavn\Anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\bhavn\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [33]:
#Displaying topic weights for each label
data_labels[1:10]

,labels,index,Topic 2,Topic 1,Topic 3,Topic 0
1,"['Sky', 'Darkness', 'Room', 'Adventure game', ...",1,0.068813,0.799203,0.065936,0.066047
2,"['Cool', 'Fashion', 'Jeans', 'Sitting', 'Denim...",2,0.330438,0.605146,0.032455,0.031960
3,"['Hair', 'Face', 'Hairstyle', 'Eyebrow', 'Fore...",3,0.022968,0.022796,0.022786,0.931450
4,"['Snapshot', 'Standing', 'Hand', 'Arm', 'Human...",4,0.027386,0.605992,0.025317,0.341305
5,"['Clothing', 'Fashion', 'Outerwear', 'Beige', ...",5,0.304825,0.028842,0.637864,0.028469
6,"['People in nature', 'Photograph', 'Black-and-...",6,0.019729,0.940877,0.019588,0.019805
7,"['Photograph', 'Standing', 'People', 'Suit', '...",7,0.246244,0.719558,0.017329,0.016870
8,"['Cool', 'Human', 'Font', 'Fur', 'Outerwear', ...",8,0.365791,0.493547,0.022513,0.118149
9,"['Photograph', 'Clothing', 'Formal wear', 'Sta...",9,0.354012,0.606579,0.020094,0.019315


In [34]:
# This bit gives out the best topic match for each document and the corresponding score
def format_topics_sentences(lda_model=None, corpus=bow_corpus, texts=processed_docs):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(lda_model[bow_corpus]):
        row = row_list[0] if lda_model.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = lda_model.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(lda_model, corpus=bow_corpus, texts=processed_docs)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,2.0,0.9378,"fashion, leg, photography, clothe, footwear, o...","[clothe, fashion, outerwear, fur, street, fash..."
1,1,1.0,0.7992,"black, photography, white, fashion, blue, mono...","[sky, darkness, room, adventure, game, music, ..."
2,2,1.0,0.6052,"black, photography, white, fashion, blue, mono...","[cool, fashion, jeans, sit, denim, shoe, style]"
3,3,0.0,0.9314,"hair, beauty, hairstyle, model, skin, long, fa...","[hair, face, hairstyle, eyebrow, forehead, chi..."
4,4,1.0,0.6060,"black, photography, white, fashion, blue, mono...","[snapshot, stand, hand, arm, human, photograph..."
5,5,3.0,0.6378,"fashion, clothe, outerwear, sleeve, neck, dres...","[clothe, fashion, outerwear, beige, fashion, d..."
6,6,1.0,0.9409,"black, photography, white, fashion, blue, mono...","[people, nature, photograph, black, white, sta..."
7,7,1.0,0.7196,"black, photography, white, fashion, blue, mono...","[photograph, stand, people, suit, black, white..."
8,8,1.0,0.4936,"black, photography, white, fashion, blue, mono...","[cool, human, font, fur, outerwear, textile, a..."
9,9,1.0,0.6066,"black, photography, white, fashion, blue, mono...","[photograph, clothe, formal, wear, stand, suit..."


In [35]:
data_labels['image_URL'] = posts_df['image_url']

cols = ['image_URL','labels','index','Topic 1','Topic 2','Topic 3']
data_labels[cols].to_csv('topics_and_weights.csv')

C:\Users\bhavn\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Engagement scores vs Topic Weights 

In [36]:
data_labels['engagement_score'] = posts_df['engagement_score']
data_labels.engagement_score.quantile([0.25,0.5,0.75])

C:\Users\bhavn\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0.25    0.095768
0.50    0.132158
0.75    0.198502
Name: engagement_score, dtype: float64

In [37]:
#Splitting the data into two quartiles for the lowest and highest engageent scores
quart_25 = data_labels[data_labels['engagement_score']<0.045992] #lowest
quart_75 = data_labels[data_labels['engagement_score']>0.097723] #highest

#Average of topic weights in each of these quartiles
avg_low = pd.DataFrame(quart_25[['Topic 0','Topic 1','Topic 2','Topic 3']].mean())
avg_low = avg_low.transpose()

avg_high = pd.DataFrame(quart_75[['Topic 0','Topic 1','Topic 2','Topic 3']].mean())
avg_high = avg_high.transpose()

In [38]:
engagement_scores_high_and_low = avg_low.append(avg_high,ignore_index=True)
engagement_scores_high_and_low['Engagement_levels'] = ['Low Engagement','High Engagement']
engagement_scores_high_and_low.set_index('Engagement_levels',inplace=True)
engagement_scores_high_and_low#.rename(columns = {"Topic 0": "A", "Topic 1": "B", "Topic 2": "C", "Topic 3": "D"})

,Topic 0,Topic 1,Topic 2,Topic 3
Engagement_levels,,,,
Low Engagement,0.041012,0.216785,0.083103,0.659099
High Engagement,0.218818,0.242639,0.299544,0.238999


### LDA Visualisation

In [39]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.071323 -0.003172       1        1  29.642672
1      0.098979 -0.139733       2        1  25.848961
0     -0.272863  0.001597       3        1  22.458757
3      0.102561  0.141308       4        1  22.049606, topic_info=         Term        Freq       Total Category  logprob  loglift
24       hair  151.000000  151.000000  Default  30.0000  30.0000
84        leg  122.000000  122.000000  Default  29.0000  29.0000
81     sleeve   72.000000   72.000000  Default  28.0000  28.0000
25  hairstyle   62.000000   62.000000  Default  27.0000  27.0000
37      black  132.000000  132.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
35      model   26.815678  131.706573   Topic4  -3.7579  -0.0797
42      white   29.005917  167.771255   Topic4  -3.6793  -0.2432
61      joint   17.391811   69.663803   Topic4  -4.1908   0.1242
18     beauty   19.636337  136.128830   Topic4  -4.0695  -0.4243
60   footwear   12.989051   74.981712   Topic4  -4.4827  -0.2413

[191 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
122       1  0.903019         ankle
119       2  0.855275  architecture
28        1  0.434577           arm
28        2  0.057944           arm
28        3  0.318690           arm
...     ...       ...           ...
46        4  0.230491        worker
109       1  0.170626        yellow
109       2  0.307126        yellow
109       3  0.068250        yellow
109       4  0.443626        yellow

[312 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1, 4])

## Task D

According to Task B, captions are very influential on a user's engagement with a post and carry more weight than the topics depicted within the image. Zara will be able to increase user response to posts by focusing on creating engaging captions. Then based on Task C, Zara should post pictures that mostly belong in Topic 2. Topic 2 includes labels such as 'fashion', 'leg', 'photography', 'clothe', 'footwear', 'outerwear', 'joint', 'shoulder', 'human',  and 'sit' suggesting that Zara post more about relaxing, summer outerwear. Some examples may include off the shoulder tops, bikinis, shorts, and trendy footwear.  To also increase engagement, Zara should reduce the amount of posts that are in Topic 3 which include the labels 'fashion', 'clothe', 'outerwear', 'sleeve', 'neck', 'dress', 'shoulder', 'design', 'shirt',  and 'pattern'. 

In conclusion, Zara can increase engagement on their posts by creating engaging captions with pictures of relaxing summer clothes. 